### [AI Platform Prediction overview](https://cloud.google.com/ai-platform/prediction/docs/overview)

In [ ]:
# Importing the required modules
import glob
import os 

import tensorflow as tf
from oauth2client.client import GoogleCredentials
from googleapiclient import discovery
from googleapiclient import errors
from google.cloud import storage

In [ ]:
"""
When you want to use the Google APIs Client Library for Python to call the AI Platform Prediction REST APIs in your code, 
you must import its package and the OAuth2 package. 
For most standard uses of AI Platform Prediction you only need to import specific modules
"""

In [ ]:
# Set path to credentials file as environment variables
currentDirectory = os.getcwd()
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.path.join(currentDirectory, 'toucanassistant-55361b918986.json')


In [ ]:
# Bucket Storage
BUCKET_NAME_2="toucan-assistant-bucket-2"
REGION_EU_WEST1="europe-west1"


AI Platform Prediction online prediction is currently available in the following regions:
  * us-central1
  * us-east1
  * us-east4
  * asia-northeast1
  * europe-west1
  
We have <b>europe-west3</b> in bucket-1, and we have <b>europe-west1</b> n bucket-2

In [ ]:
# Set job name and job folder as environment variables
os.environ['JOB_NAME']="my_second_keras_job"
os.environ['JOB_DIR']="gs://$BUCKET_NAME/keras-job-dir-2"

### [Cloud Storage Client Libraries](https://cloud.google.com/storage/docs/reference/libraries?hl=pl#client-libraries-install-python)

Google Cloud Client Libraries use our latest client library model and are our recommended option for accessing Cloud APIs programmatically, where available. Cloud Client Libraries:

  * Provide idiomatic, generated or hand-written code in each language, making the Cloud API simple and intuitive to use.
  * Handle all the low-level details of communication with the server, including authenticating with Google.
  * Can be installed using familiar package management tools such as npm and pip.
  * In some cases, give you performance benefits by using gRPC. 

### Cloud IAM:
  * [Cloud Identity and Access Management](https://cloud.google.com/storage/docs/access-control/iam)
  * GCP user must have proper role to create buckets (in this case the role is StorageAdmin)
  * [Cloud IAM roles for Cloud Storage](https://cloud.google.com/storage/docs/access-control/iam-roles) roles
  * You can't create bucket with the name that exists


### Working with buckets

In [ ]:
# The name for the new bucket
bucket_name_3 = "toucan-assistant-bucket-3"

In [ ]:
# Creates the new bucket in proper location
def create_bucket_in_location(name_of_bucket, location_of_bucket):
    # Instantiates a client
    storage_client = storage.Client()
    # https://stackoverflow.com/questions/54992263/creating-bucket-in-google-cloud-storage-in-custom-location
    # https://stackoverflow.com/questions/42576366/google-cloud-storage-python-api-create-bucket-in-specified-location
    # bucket name must be unique !!!
    custom_bucket = storage_client.bucket(name_of_bucket)
    custom_bucket.create(location=location_of_bucket)
    print(f'Bucket {name_of_bucket} was created!')

create_bucket_in_location(bucket_name_3, REGION_EU_WEST1)

In [ ]:
# Deleting bucket
# Be careful or set force=False
# force=True deletes non-empty folders !!!
def delete_bucket(name_of_bucket):
    storage_client = storage.Client()
    custom_bucket = storage_client.get_bucket(name_of_bucket)
    custom_bucket.delete(force=True)
    print(f'Bucket {name_of_bucket} was deleted!')
    
delete_bucket(bucket_name_3)

In [ ]:
# Function to copying file 
def upload_file(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # bucket_name = "your-bucket-name"
    # source_file_name = "local/path/to/file"
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        "File {} uploaded to {}.".format(
            source_file_name, destination_blob_name
        )
    )

In [ ]:
# Test on file
# Files are overwritten !!!
file_to_copy = 'requirements.txt'
file_blob_name = 'copied_requirements.txt'
upload_file(bucket_name_3, file_to_copy, file_blob_name)

In [ ]:
# https://hackersandslackers.com/manage-files-in-google-cloud-storage-with-python/
def upload_folder(bucketName, localFolder):
    """Upload files to GCP bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucketName)

    files = [f for f in os.listdir(localFolder) if os.path.isfile(os.path.join(localFolder, f))]
    for file in files:
        localFile = localFolder + '/' + file
        blob = bucket.blob(localFolder + '/' + file)
        blob.upload_from_filename(localFile)
    return f'Uploaded {files} to "{bucketName}" bucket.'

In [ ]:
def upload_local_directory_to_gcs(local_path, bucketName, gcs_path):
    assert os.path.isdir(local_path)
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucketName)
    
    for local_file in glob.glob(local_path + '/**'):
        if not os.path.isfile(local_file):
           upload_local_directory_to_gcs(local_file, bucket, gcs_path + "/" + os.path.basename(local_file))
        else:
            remote_path = os.path.join(gcs_path, local_file[1 + len(local_path):])
            blob = bucket.blob(remote_path)
            blob.upload_from_filename(local_file)
    print(f'Folder {local_path} uploaded to bucket {bucketName}')

In [ ]:
# Test on folder
folder_to_copy = 'keras_export'
folder_blob_name = 'copied_trainer'
#upload_folder(bucket_name_3, folder_to_copy)
upload_local_directory_to_gcs(folder_to_copy, bucket_name_3, folder_blob_name)

### 1st step - [Exporting saved models for prediction](https://cloud.google.com/ai-platform/prediction/docs/exporting-savedmodel-for-prediction)

<p>A SavedModel is TensorFlow's <b>recommended</b> format for saving models, and it is the required format for deploying trained TensorFlow models on AI Platform Prediction.</p>
<p>Exporting your trained model as a SavedModel saves your training graph with its assets, variables and metadata in a format that AI Platform Prediction can consume and restore for predictions.

After exporting a SavedModel, you have a SavedModel directory that contains the following:</p>

  * your training graph(s), saved in SavedModel protocol buffers
  * external files, called assets
  * variables, which are saved as checkpoint files

<p>When you deploy your SavedModel to AI Platform Prediction, you must include the <b>entire SavedModel directory</b>, not just the SavedModel protocol buffer file that contains your graph and its metadata. This file usually has an extension of either .pb or .pbtxt.</p>

<p>The SavedModel allows you to save multiple versions of a graph that share the same assets and variables (or checkpoints). For example, you may want to develop two versions of the same graph: one to run on CPUs, and another to run on GPUs.</p>

If you have used Keras for training, use [tf.keras.Model.save](https://www.tensorflow.org/guide/keras/save_and_serialize#export_to_savedmodel) to export a SavedModel


In [ ]:
JOB_DIR="gs://$BUCKET_NAME/keras-job-dir-2"
JOB_DIR = os.getenv('JOB_DIR')
# Export the model to a SavedModel directory in Cloud Storage


### 2nd step -  [Deploying models](https://cloud.google.com/ai-platform/prediction/docs/deploying-models#rest-api_1)

In order to <b>deploy your trained model</b> on AI Platform Prediction, you must:

  * Upload your saved model to a Cloud Storage bucket.
  * Create an AI Platform Prediction [model](https://cloud.google.com/ai-platform/prediction/docs/reference/rest/v1/projects.models#Model) resource.
  * Create an AI Platform Prediction [version](https://cloud.google.com/ai-platform/prediction/docs/reference/rest/v1/projects.models.versions) resource, specifying the Cloud Storage path to your saved model.

### 3rd step -  [Getting online predictions](https://cloud.google.com/ai-platform/prediction/docs/online-predict#rest-api)


In order to request predictions, you must first:

 * Export your trained model as one or more model artifacts that can be deployed to AI Platform Prediction.

 * Deploy your trained model to AI Platform Prediction by creating a model resource and version.

### How to use the [Google APIs Client Library for Python](https://cloud.google.com/ai-platform/prediction/docs/python-client-library?hl=sr) to call the AI Platform Prediction REST APIs in your Python applications